# <div align="center" style="color:rgb(51, 255, 228);">PSI and CSI Calculations</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
sq = """drop table if exists  `dap_ds_poweruser_playground.F_CSI_TT_NEW`;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a3e3bce4-78c8-40b6-9e2e-c81158247a39>

# CIC Score CIS

In [3]:
sq = """
with cicscorebase as 
(select 
digitalLoanAccountId,
FORMAT_DATE('%Y-%m', ln_appln_submit_datetime) Application_month,
FORMAT_DATE('%F', DATE_TRUNC(ln_appln_submit_datetime, WEEK(MONDAY))) as Appl_week_start_date,
EXTRACT(WEEK(MONDAY) FROM ln_appln_submit_datetime) as Appl_week_number,
ln_user_type,
ln_loan_type,
ln_prod_type, 
ln_os_type,
cic_hit_flag,
cic_score,
case when date_trunc(ln_appln_submit_datetime, day) between '2024-06-01' and '2024-09-30' then 'Train'
       when date_trunc(ln_appln_submit_datetime, day) >= '2024-10-01' then 'Test'
       Else 'Other' end dataselection,
case when cic_Personal_Loans_granted_contracts_amt_24M is null then 'g. missing'
     when cic_Personal_Loans_granted_contracts_amt_24M between 0.999 and 5650.6 then 'a. 0.999-5650.6'
     when cic_Personal_Loans_granted_contracts_amt_24M between 5650.7 and 12447.8 then 'b. 5650.7-12447.8'
     when cic_Personal_Loans_granted_contracts_amt_24M between 12447.9 and 24000.0 then 'c. 12447.9-24000.0'
     when cic_Personal_Loans_granted_contracts_amt_24M between 24000.1 and 50500.0 then 'd. 24000.1-50500'
     when cic_Personal_Loans_granted_contracts_amt_24M between 50500.1 and 2738545.0 then 'e. 50500.1-2738545.0'
     when cic_Personal_Loans_granted_contracts_amt_24M > 2738545.0 then 'f. >2738545.0' end cic_Personal_Loans_granted_contracts_amt_24M_bin,
case when cic_cnt_active_contracts is null then 'd. missing'
     when cic_cnt_active_contracts between 0.999 and 2.0 then 'a. 0.999-2.0'
     when cic_cnt_active_contracts between 2.1 and 88.0 then 'b. 2.1-88.0'
     when cic_cnt_active_contracts > 88.0 then 'c. >88.0' end cic_cnt_active_contracts_bin,
case when cic_vel_contract_nongranted_cnt_12on24 is null then 'd. missing'
     when cic_vel_contract_nongranted_cnt_12on24 between 0.285 and 1.994 then 'a. 0.285-1.994'
     when cic_vel_contract_nongranted_cnt_12on24 between 1.995 and 2.012 then 'b. 1.995-2.012'
     when cic_vel_contract_nongranted_cnt_12on24 > 2.012 then 'c. >2.012' end cic_vel_contract_nongranted_cnt_12on24_bin,
case when cic_days_since_last_inquiry is null then 'g. missing'
     when cic_days_since_last_inquiry between -0.001 and 10.0 then 'a. -0.001-10.0'
     when cic_days_since_last_inquiry between 10.1 and 117.0 then 'b. 10.1-117.0'
     when cic_days_since_last_inquiry between 117.1 and 281.0 then 'c. 117.1-281.0'
     when cic_days_since_last_inquiry between 281.1 and 832.0 then 'd. 281.1-832.0'
     when cic_days_since_last_inquiry between 832.1 and 10844.0 then 'e. 832.1-10844.0'
     when cic_days_since_last_inquiry > 10844.0 then 'f. >10844.0' end cic_days_since_last_inquiry_bin,
case when cic_max_amt_granted_24M is null then 'g. missing'
     when cic_max_amt_granted_24M between -0.001 and 5394.2 then 'a. -0.001-5394.2'
     when cic_max_amt_granted_24M between 5394.3 and 10502.4 then 'b. 5394.3-10502.4'
     when cic_max_amt_granted_24M between 10502.5 and 20000.0 then 'c. 10502.5-20000.0'
     when cic_max_amt_granted_24M between 20000.1 and 40000.0 then 'd. 20000.1-40000.0'
     when cic_max_amt_granted_24M between 40000.1 and 8000000.0 then 'e. 40000.1-8000000.0'
     when cic_max_amt_granted_24M > 8000000.0 then 'f. >8000000.0' end cic_max_amt_granted_24M_bin
from risk_mart.sil_risk_ds_master_20230101_20250309
where cic_called_flag = 1
and date_trunc(ln_appln_submit_datetime, day) >= '2024-06-01'
)
select * from cicscorebase
"""

cicscoredf = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID c858c6ac-e7fa-4ab2-a53a-bb14342c1b85 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [4]:
print(f"The shape of the df for cic score csi is {cicscoredf.shape}")

The shape of the df for cic score csi is (235305, 16)


In [5]:
cicscoredf.head()

,digitalLoanAccountId,Application_month,Appl_week_start_date,Appl_week_number,ln_user_type,ln_loan_type,ln_prod_type,ln_os_type,cic_hit_flag,cic_score,dataselection,cic_Personal_Loans_granted_contracts_amt_24M_bin,cic_cnt_active_contracts_bin,cic_vel_contract_nongranted_cnt_12on24_bin,cic_days_since_last_inquiry_bin,cic_max_amt_granted_24M_bin
0,076f3b5a-78c4-460f-b2e6-9a795a5ffa23,2025-01,2024-12-30,0,3_Applied_Not_Disbursed,SIL Competitor,Mall,Android,<NA>,NaN,Test,g. missing,d. missing,d. missing,g. missing,g. missing
1,1357b77d-c4a5-47a3-b64a-b0a54c98e1df,2025-01,2024-12-30,0,3_Applied_Not_Disbursed,SIL-Instore,Mall,Android,<NA>,NaN,Test,g. missing,d. missing,d. missing,g. missing,g. missing
2,8fa77a96-b572-4307-b371-58a1d74f37eb,2025-01,2024-12-30,0,3_Applied_Not_Disbursed,SIL-Instore,Mall,Android,<NA>,NaN,Test,g. missing,d. missing,d. missing,g. missing,g. missing
3,f84f2822-3e4a-422b-b9fe-20292a3ded90,2025-01,2024-12-30,0,3_Applied_Not_Disbursed,SIL ZERO,Mall,Android,<NA>,NaN,Test,g. missing,d. missing,d. missing,g. missing,g. missing
4,e11070e2-55d4-4377-be9e-c35290c5776d,2025-01,2024-12-30,0,3_Applied_Not_Disbursed,SIL-Instore,Appliance,Android,0,NaN,Test,g. missing,d. missing,d. missing,g. missing,g. missing


In [6]:
cicscoredf.columns.values

array(['digitalLoanAccountId', 'Application_month',
       'Appl_week_start_date', 'Appl_week_number', 'ln_user_type',
       'ln_loan_type', 'ln_prod_type', 'ln_os_type', 'cic_hit_flag',
       'cic_score', 'dataselection',
       'cic_Personal_Loans_granted_contracts_amt_24M_bin',
       'cic_cnt_active_contracts_bin',
       'cic_vel_contract_nongranted_cnt_12on24_bin',
       'cic_days_since_last_inquiry_bin', 'cic_max_amt_granted_24M_bin'],
      dtype=object)

# For Overall CSI calculation

In [7]:
# import pandas as pd
# import numpy as np
# from datetime import datetime

# def calculate_categorical_psi(train_dist, test_dist):
#     """
#     Calculate PSI for categorical features.
    
#     Args:
#         train_dist: Distribution of categories in training set
#         test_dist: Distribution of categories in test set
    
#     Returns:
#         float: PSI value
#     """
#     # Ensure both distributions have the same categories
#     all_categories = set(train_dist.index) | set(test_dist.index)
    
#     # Align distributions
#     train_dist_aligned = train_dist.reindex(all_categories, fill_value=0.0001)  # Small value to avoid division by zero
#     test_dist_aligned = test_dist.reindex(all_categories, fill_value=0.0001)
    
#     # Calculate PSI
#     psi_values = (test_dist_aligned - train_dist_aligned) * np.log(test_dist_aligned / train_dist_aligned)
#     return psi_values.sum()

# def calculate_feature_psi(df, feature_list):
#     """
#     Calculate PSI for multiple categorical features.
    
#     Args:
#         df: DataFrame containing the data
#         feature_list: List of feature names to calculate PSI for
    
#     Returns:
#         DataFrame: PSI results for each feature by month
#     """
#     # Make a copy to avoid modifying the original DataFrame
#     df_copy = df.copy()
    
#     # Separate train and test data
#     train_df = df_copy[df_copy['dataselection'] == 'Train']
#     test_df = df_copy[df_copy['dataselection'] == 'Test']
    
#     # Handle Application_month based on its type
#     if isinstance(df_copy['Application_month'].iloc[0], str):
#         # If it's a string in format 'YYYY-MM-DD', extract just 'YYYY-MM'
#         last_train_month_str = str(train_df['Application_month'].max())
#         if len(last_train_month_str) >= 7:  # Ensure we have at least YYYY-MM
#             last_train_month_str = last_train_month_str[:7]  # Extract YYYY-MM part
#     else:
#         # If it's already a datetime object
#         try:
#             last_train_month = pd.to_datetime(train_df['Application_month'].max())
#             last_train_month_str = last_train_month.strftime('%Y-%m')
#         except:
#             # Fallback if conversion fails
#             last_train_month_str = str(train_df['Application_month'].max())
    
#     # Store PSI results
#     psi_results = []
    
#     # Calculate distribution for each feature in the training set
#     feature_train_distributions = {}
#     for feature in feature_list:
#         # Calculate distribution, excluding missing values
#         counts = train_df[feature].value_counts(dropna=True)
#         if len(counts) > 0:  # Check if we have any non-null values
#             distribution = counts / counts.sum()
#             feature_train_distributions[feature] = distribution
            
#             # Add the train set PSI (which is 0 by definition when comparing with itself)
#             psi_results.append({
#                 'Month': last_train_month_str,
#                 'feature': feature,
#                 'DateCategory': 'a_Training',  # Added 'a_' prefix for sorting
#                 'csivalues': 0.0,  # PSI of training set against itself is 0
#                 'account_count': train_df['digitalLoanAccountId'].nunique()
#             })
    
#     # Get unique months from test set and sort them
#     test_months = sorted(test_df['Application_month'].unique())
    
#     # Create mapping of months to prefixed labels (b, c, d, etc.)
#     prefix_map = {}
#     for i, month in enumerate(test_months):
#         month_str = str(month)
#         if isinstance(month, str) and len(month_str) >= 7:
#             month_str = month_str[:7]  # Extract YYYY-MM part
        
#         # Use letters b, c, d, etc. for subsequent months (a is reserved for Training)
#         prefix = chr(98 + i)  # ASCII: b=98, c=99, etc.
#         prefix_map[month] = f"{prefix}_{month_str}"
    
#     # Calculate monthly PSI for each feature in the test set
#     for month in test_months:
#         original_month_str = str(month)
#         if isinstance(month, str) and len(original_month_str) >= 7:
#             original_month_str = original_month_str[:7]  # Extract YYYY-MM part
        
#         # Use the prefixed month string for sorting
#         month_str = prefix_map[month]
        
#         month_df = test_df[test_df['Application_month'] == month]
        
#         if not month_df.empty:
#             month_accounts = month_df['digitalLoanAccountId'].nunique()
            
#             for feature in feature_list:
#                 if feature in feature_train_distributions:  # Check if we have a training distribution
#                     # Calculate distribution for the current month
#                     counts = month_df[feature].value_counts(dropna=True)
#                     if len(counts) > 0:  # Ensure we have data for this feature in this month
#                         distribution = counts / counts.sum()
                        
#                         # Calculate PSI comparing with training distribution
#                         month_psi = calculate_categorical_psi(
#                             feature_train_distributions[feature],
#                             distribution
#                         )
                        
#                         psi_results.append({
#                             'Month': original_month_str,  # Original month string for display
#                             'MonthSortKey': month_str,    # Prefixed month string for sorting
#                             'feature': feature,
#                             'DateCategory': 'b_Monthly',  # Added 'b_' prefix for sorting
#                             'csivalues': month_psi,
#                             'account_count': month_accounts
#                         })
    
#     # Create the output DataFrame
#     return pd.DataFrame(psi_results)

# # Example usage
# feature_list = [
#     'cic_Personal_Loans_granted_contracts_amt_24M_bin',
#     'cic_cnt_active_contracts_bin',
#     'cic_vel_contract_nongranted_cnt_12on24_bin',
#     'cic_days_since_last_inquiry_bin',
#     'cic_max_amt_granted_24M_bin'
# ]

# # Calculate PSI for all features
# features_psi_df = calculate_feature_psi(cicscoredf, feature_list)

# # Combine with s_apps_score results if they exist
# try:
#     # First ensure the s_apps_score_output_df has the same structure
#     if 'MonthSortKey' not in s_apps_score_output_df.columns:
#         s_apps_score_output_df['MonthSortKey'] = s_apps_score_output_df['Month']
#         # Update DateCategory with prefix
#         s_apps_score_output_df['DateCategory'] = s_apps_score_output_df['DateCategory'].apply(
#             lambda x: 'a_Training' if x == 'Training' else 'b_Monthly'
#         )
    
#     # Check if s_apps_score_output_df exists
#     all_psi_results = pd.concat([s_apps_score_output_df, features_psi_df], ignore_index=True)
    
#     # Replace 'scorename' with 'feature' for consistency
#     if 'scorename' in all_psi_results.columns:
#         all_psi_results['feature'] = all_psi_results['feature'].fillna(all_psi_results['scorename'])
#         all_psi_results = all_psi_results.drop('scorename', axis=1)
# except NameError:
#     # If s_apps_score_output_df doesn't exist, just use features_psi_df
#     all_psi_results = features_psi_df

# # Sort by MonthSortKey (which includes prefix) and feature
# if 'MonthSortKey' in all_psi_results.columns:
#     all_psi_results = all_psi_results.sort_values(['MonthSortKey', 'feature'])
# else:
#     all_psi_results = all_psi_results.sort_values(['Month', 'feature'])

# # Save the results
# all_psi_results.to_csv('feature_psi_results.csv', index=False)

# # Display the first few rows 
# print(all_psi_results.head())

# # Optional: Display a pivot table view for better visualization
# pivot_df = all_psi_results.pivot_table(
#     index=['feature'], 
#     columns=['MonthSortKey'], 
#     values='csivalues',
#     aggfunc='first'
# )
# print("\nPivot table view of CSI values:")
# print(pivot_df)

In [8]:
# all_psi_results

In [9]:
# all_psi_results['Month'] = all_psi_results['Month'].replace('2024-09', '2024-06-2024-09')
# all_psi_results['MonthSortKey'] = all_psi_results['MonthSortKey'].fillna('a_2024-06-2024-09')
# all_psi_results['Month'] = all_psi_results['Month'].apply(lambda x: x.split(' 00:00:00')[0] if'00:00:00' in x else x)

In [10]:
# all_psi_results['scorename'] = 'CIC_Score'
# all_psi_results['Modelname'] = 'SIL CIC Model'
# all_psi_results['Description'] = 'Train period from 2024-06 to 2024-09'

In [11]:
# all_psi_results.dtypes

In [12]:
# all_psi_results.sort_values(by='MonthSortKey')

In [13]:
# dataset_id = 'dap_ds_poweruser_playground'
# table_id = 'F_CSI_TT_NEW'
# # Define the table schema as per your DataFrame columns
# schema = [
#     bigquery.SchemaField("Month", "string"),
#     bigquery.SchemaField("feature", "string"),
#     bigquery.SchemaField("DateCategory", "string"),
#     bigquery.SchemaField("csivalues", "float64"),
#     bigquery.SchemaField("account_count", "int64"),
#     bigquery.SchemaField("MonthSortKey", "string"),
#     bigquery.SchemaField("scorename", "string"),
#     bigquery.SchemaField("Modelname", "string"),
#     bigquery.SchemaField("Description", "string")
#     ]
# # Create the dataset reference
# dataset_ref = client.dataset(dataset_id)
# # Define the table reference
# table_ref = dataset_ref.table(table_id)
# # Configure the job to overwrite the table if it already exists
# job_config = bigquery.LoadJobConfig(schema = schema)
# # Load the DataFrame into BigQuery
# job = client.load_table_from_dataframe(all_psi_results, table_ref, job_config=job_config)
# # Wait for the job to complete
# job.result()
# print(f"Table {table_id} created in dataset {dataset_id}.")

**Logic :-** <br>
`1. Calculate the CSI the same way as we have calculated PSI but for top 5 features of a Model.`<br>
`2. We will calculate it for overall, then for each categories of user_type, prod_type and os_type.`<br>

In [14]:
import pandas as pd
import numpy as np
from datetime import datetime

def calculate_categorical_psi(train_dist, test_dist):
    """
    Calculate PSI for categorical features.
    
    Args:
        train_dist: Distribution of categories in training set
        test_dist: Distribution of categories in test set
    
    Returns:
        float: PSI value
    """
    # Ensure both distributions have the same categories
    all_categories = set(train_dist.index) | set(test_dist.index)
    
    # Align distributions
    train_dist_aligned = train_dist.reindex(all_categories, fill_value=0.0001)  # Small value to avoid division by zero
    test_dist_aligned = test_dist.reindex(all_categories, fill_value=0.0001)
    
    # Calculate PSI
    psi_values = (test_dist_aligned - train_dist_aligned) * np.log(test_dist_aligned / train_dist_aligned)
    return psi_values.sum()

def calculate_segmented_psi(df, feature_list, segment_columns=None):
    """
    Calculate PSI for multiple categorical features, overall and by segments.
    
    Args:
        df: DataFrame containing the data
        feature_list: List of feature names to calculate PSI for
        segment_columns: List of columns to segment by (e.g., ['ln_user_type', 'ln_os_type'])
    
    Returns:
        DataFrame: PSI results for each feature by month and segment
    """
    # Initialize results list
    all_results = []
    
    # If no segment columns are provided, use an empty list
    if segment_columns is None:
        segment_columns = []
    
    # First, calculate overall PSI
    overall_results = calculate_feature_psi(df, feature_list)
    overall_results['segment_type'] = 'Overall'
    overall_results['segment_value'] = 'All'
    all_results.append(overall_results)
    
    # Then calculate PSI for each segment column
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            print(f"Warning: {segment_col} not found in DataFrame. Skipping.")
            continue
        
        # Get unique segment values
        segment_values = df[segment_col].dropna().unique()
        
        for segment_val in segment_values:
            # Filter data for this segment
            segment_df = df[df[segment_col] == segment_val]
            
            # Skip if not enough data
            if len(segment_df) < 50:  # Arbitrary threshold
                print(f"Skipping {segment_col}={segment_val} due to insufficient data ({len(segment_df)} rows).")
                continue
                
            # Calculate PSI for this segment
            try:
                segment_results = calculate_feature_psi(segment_df, feature_list)
                segment_results['segment_type'] = segment_col
                segment_results['segment_value'] = segment_val
                all_results.append(segment_results)
            except Exception as e:
                print(f"Error calculating PSI for {segment_col}={segment_val}: {e}")
    
    # Combine all results
    if all_results:
        combined_results = pd.concat(all_results, ignore_index=True)
        return combined_results
    else:
        return pd.DataFrame()

def calculate_feature_psi(df, feature_list):
    """
    Calculate PSI for multiple categorical features.
    
    Args:
        df: DataFrame containing the data
        feature_list: List of feature names to calculate PSI for
    
    Returns:
        DataFrame: PSI results for each feature by month
    """
    # Make a copy to avoid modifying the original DataFrame
    df_copy = df.copy()
    
    # Separate train and test data
    train_df = df_copy[df_copy['dataselection'] == 'Train']
    test_df = df_copy[df_copy['dataselection'] == 'Test']
    
    # Skip if either dataset is empty
    if train_df.empty or test_df.empty:
        print("Warning: Either train or test dataset is empty. Skipping PSI calculation.")
        return pd.DataFrame()
    
    # Handle Application_month based on its type
    if isinstance(df_copy['Application_month'].iloc[0], str):
        # If it's a string in format 'YYYY-MM-DD', extract just 'YYYY-MM'
        last_train_month_str = str(train_df['Application_month'].max())
        if len(last_train_month_str) >= 7:  # Ensure we have at least YYYY-MM
            last_train_month_str = last_train_month_str[:7]  # Extract YYYY-MM part
    else:
        # If it's already a datetime object
        try:
            last_train_month = pd.to_datetime(train_df['Application_month'].max())
            last_train_month_str = last_train_month.strftime('%Y-%m')
        except:
            # Fallback if conversion fails
            last_train_month_str = str(train_df['Application_month'].max())
    
    # Store PSI results
    psi_results = []
    
    # Calculate distribution for each feature in the training set
    feature_train_distributions = {}
    for feature in feature_list:
        if feature not in train_df.columns:
            print(f"Warning: Feature {feature} not found in training data. Skipping.")
            continue
            
        # Calculate distribution, excluding missing values
        counts = train_df[feature].value_counts(dropna=True)
        if len(counts) > 0:  # Check if we have any non-null values
            distribution = counts / counts.sum()
            feature_train_distributions[feature] = distribution
            
            # Add the train set PSI (which is 0 by definition when comparing with itself)
            psi_results.append({
                'Month': last_train_month_str,
                'feature': feature,
                'DateCategory': 'a_Training',  # Added 'a_' prefix for sorting
                'csivalues': 0.0,  # PSI of training set against itself is 0
                'account_count': train_df['digitalLoanAccountId'].nunique()
            })
    
    # Get unique months from test set and sort them
    test_months = sorted(test_df['Application_month'].unique())
    
    # Create mapping of months to prefixed labels (b, c, d, etc.)
    prefix_map = {}
    for i, month in enumerate(test_months):
        month_str = str(month)
        if isinstance(month, str) and len(month_str) >= 7:
            month_str = month_str[:7]  # Extract YYYY-MM part
        
        # Use letters b, c, d, etc. for subsequent months (a is reserved for Training)
        prefix = chr(98 + i)  # ASCII: b=98, c=99, etc.
        prefix_map[month] = f"{prefix}_{month_str}"
    
    # Calculate monthly PSI for each feature in the test set
    for month in test_months:
        original_month_str = str(month)
        if isinstance(month, str) and len(original_month_str) >= 7:
            original_month_str = original_month_str[:7]  # Extract YYYY-MM part
        
        # Use the prefixed month string for sorting
        month_str = prefix_map[month]
        
        month_df = test_df[test_df['Application_month'] == month]
        
        if not month_df.empty:
            month_accounts = month_df['digitalLoanAccountId'].nunique()
            
            for feature in feature_list:
                if feature not in month_df.columns:
                    continue
                    
                if feature in feature_train_distributions:  # Check if we have a training distribution
                    # Calculate distribution for the current month
                    counts = month_df[feature].value_counts(dropna=True)
                    if len(counts) > 0:  # Ensure we have data for this feature in this month
                        distribution = counts / counts.sum()
                        
                        # Calculate PSI comparing with training distribution
                        month_psi = calculate_categorical_psi(
                            feature_train_distributions[feature],
                            distribution
                        )
                        
                        psi_results.append({
                            'Month': original_month_str,  # Original month string for display
                            'MonthSortKey': month_str,    # Prefixed month string for sorting
                            'feature': feature,
                            'DateCategory': 'b_Monthly',  # Added 'b_' prefix for sorting
                            'csivalues': month_psi,
                            'account_count': month_accounts
                        })
    
    # Create the output DataFrame
    return pd.DataFrame(psi_results)

# Features list
feature_list = [
    'cic_Personal_Loans_granted_contracts_amt_24M_bin',
    'cic_cnt_active_contracts_bin',
    'cic_vel_contract_nongranted_cnt_12on24_bin',
    'cic_days_since_last_inquiry_bin',
    'cic_max_amt_granted_24M_bin'
]

# Define segment columns
segment_columns = ['ln_user_type', 'ln_prod_type', 'ln_os_type']  # Changed ln_os_type to ln_prod_type based on your data

# Calculate PSI for overall and by segments
all_results = calculate_segmented_psi(cicscoredf, feature_list, segment_columns)

# Combine with s_apps_score results if they exist
try:
    # First ensure the s_apps_score_output_df has the same structure
    if 'MonthSortKey' not in s_apps_score_output_df.columns:
        s_apps_score_output_df['MonthSortKey'] = s_apps_score_output_df['Month']
        # Update DateCategory with prefix
        s_apps_score_output_df['DateCategory'] = s_apps_score_output_df['DateCategory'].apply(
            lambda x: 'a_Training' if x == 'Training' else 'b_Monthly'
        )
    
    # Add segment info to s_apps_score_output_df
    s_apps_score_output_df['segment_type'] = 'Overall'
    s_apps_score_output_df['segment_value'] = 'All'
    
    # Check if s_apps_score_output_df exists
    combined_results = pd.concat([s_apps_score_output_df, all_results], ignore_index=True)
    
    # Replace 'scorename' with 'feature' for consistency
    if 'scorename' in combined_results.columns:
        combined_results['feature'] = combined_results['feature'].fillna(combined_results['scorename'])
        combined_results = combined_results.drop('scorename', axis=1)
except NameError:
    # If s_apps_score_output_df doesn't exist, just use all_results
    combined_results = all_results

# Sort by segment_type, segment_value, MonthSortKey and feature
sort_columns = ['segment_type', 'segment_value']
if 'MonthSortKey' in combined_results.columns:
    sort_columns.append('MonthSortKey')
else:
    sort_columns.append('Month')
sort_columns.append('feature')

combined_results = combined_results.sort_values(sort_columns)

# Save the results
combined_results.to_csv('segmented_feature_psi_results.csv', index=False)

# Display the first few rows
print("Sample of results:")
print(combined_results.head())

# Create pivot tables for easier analysis
print("\nGenerating pivot tables for analysis...")

# Function to create pivot table for a given segment
def create_pivot_for_segment(data, segment_type, segment_value):
    segment_data = data[(data['segment_type'] == segment_type) & 
                       (data['segment_value'] == segment_value)]
    
    # Create pivot table
    pivot = segment_data.pivot_table(
        index=['feature'], 
        columns=['MonthSortKey'] if 'MonthSortKey' in segment_data.columns else ['Month'], 
        values='csivalues',
        aggfunc='first'
    )
    
    return pivot

# Create and save pivot tables for overall and each segment
unique_segments = combined_results[['segment_type', 'segment_value']].drop_duplicates()

# Create a dictionary to store all pivot tables
pivot_tables = {}

# Generate pivot tables for each segment
for _, row in unique_segments.iterrows():
    segment_type = row['segment_type']
    segment_value = row['segment_value']
    
    # Create a key for this segment
    segment_key = f"{segment_type}_{segment_value}"
    
    # Create pivot table
    pivot = create_pivot_for_segment(combined_results, segment_type, segment_value)
    
    # Store in dictionary
    pivot_tables[segment_key] = pivot
    
    # Print some info
    print(f"\nPivot table for {segment_type} - {segment_value}:")
    print(pivot.head(3))  # Show just first 3 rows

# Save all pivot tables to Excel with multiple sheets
with pd.ExcelWriter('segmented_feature_psi_pivots.xlsx') as writer:
    for segment_key, pivot_df in pivot_tables.items():
        # Clean sheet name (Excel has 31 character limit)
        sheet_name = segment_key[:31].replace('/', '_')
        pivot_df.to_excel(writer, sheet_name=sheet_name)

print("\nAll pivot tables saved to 'segmented_feature_psi_pivots.xlsx'")

Sample of results:
     Month                                           feature DateCategory  \
5  2024-10  cic_Personal_Loans_granted_contracts_amt_24M_bin    b_Monthly   
6  2024-10                      cic_cnt_active_contracts_bin    b_Monthly   
8  2024-10                   cic_days_since_last_inquiry_bin    b_Monthly   
9  2024-10                       cic_max_amt_granted_24M_bin    b_Monthly   
7  2024-10        cic_vel_contract_nongranted_cnt_12on24_bin    b_Monthly   

   csivalues  account_count MonthSortKey segment_type segment_value  
5   0.005529          23490    b_2024-10      Overall           All  
6   0.003253          23490    b_2024-10      Overall           All  
8   0.009008          23490    b_2024-10      Overall           All  
9   0.011707          23490    b_2024-10      Overall           All  
7   0.000320          23490    b_2024-10      Overall           All  

Generating pivot tables for analysis...

Pivot table for Overall - All:
MonthSortKey             

In [15]:
combined_results['Month'] = combined_results['Month'].replace('2024-09', '2024-06-2024-09')
combined_results['MonthSortKey'] = combined_results['MonthSortKey'].fillna('a_2024-06-2024-09')
combined_results['Month'] = combined_results['Month'].apply(lambda x: x.split(' 00:00:00')[0] if'00:00:00' in x else x)
combined_results['scorename'] = 'CIC_Score'
combined_results['Modelname'] = 'SIL CIC Model'
combined_results['Description'] = 'Train period from 2024-06 to 2024-09'
combined_results

,Month,feature,DateCategory,csivalues,account_count,MonthSortKey,segment_type,segment_value,scorename,Modelname,Description
5,2024-10,cic_Personal_Loans_granted_contracts_amt_24M_bin,b_Monthly,0.005529,23490,b_2024-10,Overall,All,CIC_Score,SIL CIC Model,Train period from 2024-06 to 2024-09
6,2024-10,cic_cnt_active_contracts_bin,b_Monthly,0.003253,23490,b_2024-10,Overall,All,CIC_Score,SIL CIC Model,Train period from 2024-06 to 2024-09
8,2024-10,cic_days_since_last_inquiry_bin,b_Monthly,0.009008,23490,b_2024-10,Overall,All,CIC_Score,SIL CIC Model,Train period from 2024-06 to 2024-09
9,2024-10,cic_max_amt_granted_24M_bin,b_Monthly,0.011707,23490,b_2024-10,Overall,All,CIC_Score,SIL CIC Model,Train period from 2024-06 to 2024-09
7,2024-10,cic_vel_contract_nongranted_cnt_12on24_bin,b_Monthly,0.000320,23490,b_2024-10,Overall,All,CIC_Score,SIL CIC Model,Train period from 2024-06 to 2024-09
...,...,...,...,...,...,...,...,...,...,...,...
35,2024-06-2024-09,cic_Personal_Loans_granted_contracts_amt_24M_bin,a_Training,0.000000,34087,a_2024-06-2024-09,ln_user_type,3_Applied_Not_Disbursed,CIC_Score,SIL CIC Model,Train period from 2024-06 to 2024-09
36,2024-06-2024-09,cic_cnt_active_contracts_bin,a_Training,0.000000,34087,a_2024-06-2024-09,ln_user_type,3_Applied_Not_Disbursed,CIC_Score,SIL CIC Model,Train period from 2024-06 to 2024-09
38,2024-06-2024-09,cic_days_since_last_inquiry_bin,a_Training,0.000000,34087,a_2024-06-2024-09,ln_user_type,3_Applied_Not_Disbursed,CIC_Score,SIL CIC Model,Train period from 2024-06 to 2024-09
39,2024-06-2024-09,cic_max_amt_granted_24M_bin,a_Training,0.000000,34087,a_2024-06-2024-09,ln_user_type,3_Applied_Not_Disbursed,CIC_Score,SIL CIC Model,Train period from 2024-06 to 2024-09


In [16]:
combined_results.dtypes

Month             object
feature           object
DateCategory      object
csivalues        float64
account_count      int64
MonthSortKey      object
segment_type      object
segment_value     object
scorename         object
Modelname         object
Description       object
dtype: object

In [17]:
dataset_id = 'dap_ds_poweruser_playground'
table_id = 'F_CSI_TT_NEW'
# Define the table schema as per your DataFrame columns
schema = [
    bigquery.SchemaField("Month", "string"),
    bigquery.SchemaField("feature", "string"),
    bigquery.SchemaField("DateCategory", "string"),
    bigquery.SchemaField("csivalues", "float64"),
    bigquery.SchemaField("account_count", "int64"),
    bigquery.SchemaField("MonthSortKey", "string"),
    bigquery.SchemaField("segment_type", "string"),
    bigquery.SchemaField("segment_value", "string"),
    bigquery.SchemaField("scorename", "string"),
    bigquery.SchemaField("Modelname", "string"),
    bigquery.SchemaField("Description", "string")
    ]
# Create the dataset reference
dataset_ref = client.dataset(dataset_id)
# Define the table reference
table_ref = dataset_ref.table(table_id)
# Configure the job to overwrite the table if it already exists
job_config = bigquery.LoadJobConfig(schema = schema)
# Load the DataFrame into BigQuery
job = client.load_table_from_dataframe(combined_results, table_ref, job_config=job_config)
# Wait for the job to complete
job.result()
print(f"Table {table_id} created in dataset {dataset_id}.")

Table F_CSI_TT_NEW created in dataset dap_ds_poweruser_playground.


# beta_demo_score

In [18]:
sq = """
with sildemo as 
(select 
digitalLoanAccountId,
FORMAT_DATE('%Y-%m', ln_appln_submit_datetime) Application_month,
FORMAT_DATE('%F', DATE_TRUNC(ln_appln_submit_datetime, WEEK(MONDAY))) as Appl_week_start_date,
EXTRACT(WEEK(MONDAY) FROM ln_appln_submit_datetime) as Appl_week_number,
ln_user_type,
ln_loan_type,
ln_prod_type, 
ln_os_type,
beta_demo_score,
case when date_trunc(ln_appln_submit_datetime, day) between '2023-07-01' and '2024-06-30' then 'Train'
       when date_trunc(ln_appln_submit_datetime, day) >= '2024-07-01' then 'Test'
       Else 'Other' end dataselection,
case when beta_de_ln_vas_opted_flag is null then 'c. missing'
     when beta_de_ln_vas_opted_flag = '1' then 'a. 1'
     when beta_de_ln_vas_opted_flag = '0' then 'b. 0'
     end beta_de_ln_vas_opted_flag_bin,
case when beta_de_ln_doc_type_rolled is null then 'i. missing'
     when beta_de_ln_doc_type_rolled like 'Driving License' then 'a. Driving License'
     when beta_de_ln_doc_type_rolled like 'Others' then 'b. Others'
     when beta_de_ln_doc_type_rolled like 'Passport' then 'c. Passport'
     when beta_de_ln_doc_type_rolled like 'Postal ID Card' then 'd. Postal ID Card'
     when beta_de_ln_doc_type_rolled like 'Professional ID Card' then 'e. Professional ID Card'
     when beta_de_ln_doc_type_rolled like 'Social Security Card' then 'f. Social Security Card'
     when beta_de_ln_doc_type_rolled like 'UMID Card' then 'g. UMID Card'
     when beta_de_ln_doc_type_rolled like 'Voter Card' then 'h. Voter Card'
     else 'j. NA' end beta_de_ln_doc_type_rolled_bin,
case when (beta_de_ln_marital_status is null or beta_de_ln_marital_status like 'nan') then 'f. missing'
     when beta_de_ln_marital_status like 'Annulled / Separated' then 'a. Annulled / Separated'
     when beta_de_ln_marital_status like 'Married' then 'b. Married'
     when beta_de_ln_marital_status like 'Single' then 'c. Single'
     when beta_de_ln_marital_status like 'Widow / Widower' then 'd. Widow / Widower'
     when beta_de_ln_marital_status like 'With a Live-in Partner' then 'e. With a Live-in Partner'
     else 'g. NA' end beta_de_ln_marital_status_bin,
beta_de_ln_age_bin,
case when (beta_de_ln_ref1_type is null or beta_de_ln_ref1_type like 'nan') then 'g. missing'
     when beta_de_ln_ref1_type like 'Child' then 'a. Child'
     when beta_de_ln_ref1_type like 'Co-worker' then 'b. Co-worker'
     when beta_de_ln_ref1_type like 'Friend' then 'c. Friend'
     when beta_de_ln_ref1_type like 'Parent' then 'd. Parent'
     when beta_de_ln_ref1_type like 'Sibling' then 'e. Sibling'
     when beta_de_ln_ref1_type like 'Spouse' then 'f. Spouse'
     else 'h. NA' end beta_de_ln_ref1_type_bin
from risk_mart.sil_risk_ds_master_20230101_20250309
where date_trunc(ln_appln_submit_datetime, day) >= '2023-07-01'
)
select * from sildemo;
"""
sildemodf = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 70425bc1-4b66-4bea-9839-24ec76937e1e successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [19]:
print(f"The shape of the sildemodf is:\t {sildemodf.shape}")

The shape of the sildemodf is:	 (341586, 15)


In [20]:
sildemodf.head() 

,digitalLoanAccountId,Application_month,Appl_week_start_date,Appl_week_number,ln_user_type,ln_loan_type,ln_prod_type,ln_os_type,beta_demo_score,dataselection,beta_de_ln_vas_opted_flag_bin,beta_de_ln_doc_type_rolled_bin,beta_de_ln_marital_status_bin,beta_de_ln_age_bin,beta_de_ln_ref1_type_bin
0,8000ae81-7347-4f15-b10f-fdddb0ba98ac,2024-02,2024-02-12,7,3_Applied_Not_Disbursed,SIL-Instore,Mall,Android,0.058853,Train,a. 1,c. Passport,b. Married,"(39.0, 47.0]",e. Sibling
1,f756a256-7ca9-415c-b675-e3a12716534d,2023-08,2023-08-07,32,3_Applied_Not_Disbursed,SIL-Instore,Appliance,iOS,0.117847,Train,b. 0,g. UMID Card,c. Single,"(33.0, 39.0]",g. missing
2,78cf8153-22a0-4693-b065-8a0429b29dba,2023-08,2023-07-31,31,3_Applied_Not_Disbursed,SIL-Instore,Appliance,Android,0.091170,Train,b. 0,g. UMID Card,b. Married,"(33.0, 39.0]",g. missing
3,94481757-052d-4596-aba9-f20d05c2a72b,2023-08,2023-08-28,35,3_Applied_Not_Disbursed,SIL-Instore,Mall,Android,0.102641,Train,b. 0,e. Professional ID Card,c. Single,"(28.0, 33.0]",g. missing
4,c89d5cea-0216-44d2-b0af-587123e81718,2023-09,2023-08-28,35,3_Applied_Not_Disbursed,SIL-Instore,Mall,Android,0.120976,Train,b. 0,c. Passport,b. Married,"(39.0, 47.0]",g. missing


In [21]:
sildemodf.columns.values

array(['digitalLoanAccountId', 'Application_month',
       'Appl_week_start_date', 'Appl_week_number', 'ln_user_type',
       'ln_loan_type', 'ln_prod_type', 'ln_os_type', 'beta_demo_score',
       'dataselection', 'beta_de_ln_vas_opted_flag_bin',
       'beta_de_ln_doc_type_rolled_bin', 'beta_de_ln_marital_status_bin',
       'beta_de_ln_age_bin', 'beta_de_ln_ref1_type_bin'], dtype=object)

In [22]:
import pandas as pd
import numpy as np
from datetime import datetime

def calculate_categorical_psi(train_dist, test_dist):
    """
    Calculate PSI for categorical features.
    
    Args:
        train_dist: Distribution of categories in training set
        test_dist: Distribution of categories in test set
    
    Returns:
        float: PSI value
    """
    # Ensure both distributions have the same categories
    all_categories = set(train_dist.index) | set(test_dist.index)
    
    # Align distributions
    train_dist_aligned = train_dist.reindex(all_categories, fill_value=0.0001)  # Small value to avoid division by zero
    test_dist_aligned = test_dist.reindex(all_categories, fill_value=0.0001)
    
    # Calculate PSI
    psi_values = (test_dist_aligned - train_dist_aligned) * np.log(test_dist_aligned / train_dist_aligned)
    return psi_values.sum()

def calculate_segmented_psi(df, feature_list, segment_columns=None):
    """
    Calculate PSI for multiple categorical features, overall and by segments.
    
    Args:
        df: DataFrame containing the data
        feature_list: List of feature names to calculate PSI for
        segment_columns: List of columns to segment by (e.g., ['ln_user_type', 'ln_os_type'])
    
    Returns:
        DataFrame: PSI results for each feature by month and segment
    """
    # Initialize results list
    all_results = []
    
    # If no segment columns are provided, use an empty list
    if segment_columns is None:
        segment_columns = []
    
    # First, calculate overall PSI
    overall_results = calculate_feature_psi(df, feature_list)
    overall_results['segment_type'] = 'Overall'
    overall_results['segment_value'] = 'All'
    all_results.append(overall_results)
    
    # Then calculate PSI for each segment column
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            print(f"Warning: {segment_col} not found in DataFrame. Skipping.")
            continue
        
        # Get unique segment values
        segment_values = df[segment_col].dropna().unique()
        
        for segment_val in segment_values:
            # Filter data for this segment
            segment_df = df[df[segment_col] == segment_val]
            
            # Skip if not enough data
            if len(segment_df) < 50:  # Arbitrary threshold
                print(f"Skipping {segment_col}={segment_val} due to insufficient data ({len(segment_df)} rows).")
                continue
                
            # Calculate PSI for this segment
            try:
                segment_results = calculate_feature_psi(segment_df, feature_list)
                segment_results['segment_type'] = segment_col
                segment_results['segment_value'] = segment_val
                all_results.append(segment_results)
            except Exception as e:
                print(f"Error calculating PSI for {segment_col}={segment_val}: {e}")
    
    # Combine all results
    if all_results:
        combined_results = pd.concat(all_results, ignore_index=True)
        return combined_results
    else:
        return pd.DataFrame()

def calculate_feature_psi(df, feature_list):
    """
    Calculate PSI for multiple categorical features.
    
    Args:
        df: DataFrame containing the data
        feature_list: List of feature names to calculate PSI for
    
    Returns:
        DataFrame: PSI results for each feature by month
    """
    # Make a copy to avoid modifying the original DataFrame
    df_copy = df.copy()
    
    # Separate train and test data
    train_df = df_copy[df_copy['dataselection'] == 'Train']
    test_df = df_copy[df_copy['dataselection'] == 'Test']
    
    # Skip if either dataset is empty
    if train_df.empty or test_df.empty:
        print("Warning: Either train or test dataset is empty. Skipping PSI calculation.")
        return pd.DataFrame()
    
    # Handle Application_month based on its type
    if isinstance(df_copy['Application_month'].iloc[0], str):
        # If it's a string in format 'YYYY-MM-DD', extract just 'YYYY-MM'
        last_train_month_str = str(train_df['Application_month'].max())
        if len(last_train_month_str) >= 7:  # Ensure we have at least YYYY-MM
            last_train_month_str = last_train_month_str[:7]  # Extract YYYY-MM part
    else:
        # If it's already a datetime object
        try:
            last_train_month = pd.to_datetime(train_df['Application_month'].max())
            last_train_month_str = last_train_month.strftime('%Y-%m')
        except:
            # Fallback if conversion fails
            last_train_month_str = str(train_df['Application_month'].max())
    
    # Store PSI results
    psi_results = []
    
    # Calculate distribution for each feature in the training set
    feature_train_distributions = {}
    for feature in feature_list:
        if feature not in train_df.columns:
            print(f"Warning: Feature {feature} not found in training data. Skipping.")
            continue
            
        # Calculate distribution, excluding missing values
        counts = train_df[feature].value_counts(dropna=True)
        if len(counts) > 0:  # Check if we have any non-null values
            distribution = counts / counts.sum()
            feature_train_distributions[feature] = distribution
            
            # Add the train set PSI (which is 0 by definition when comparing with itself)
            psi_results.append({
                'Month': last_train_month_str,
                'feature': feature,
                'DateCategory': 'a_Training',  # Added 'a_' prefix for sorting
                'csivalues': 0.0,  # PSI of training set against itself is 0
                'account_count': train_df['digitalLoanAccountId'].nunique()
            })
    
    # Get unique months from test set and sort them
    test_months = sorted(test_df['Application_month'].unique())
    
    # Create mapping of months to prefixed labels (b, c, d, etc.)
    prefix_map = {}
    for i, month in enumerate(test_months):
        month_str = str(month)
        if isinstance(month, str) and len(month_str) >= 7:
            month_str = month_str[:7]  # Extract YYYY-MM part
        
        # Use letters b, c, d, etc. for subsequent months (a is reserved for Training)
        prefix = chr(98 + i)  # ASCII: b=98, c=99, etc.
        prefix_map[month] = f"{prefix}_{month_str}"
    
    # Calculate monthly PSI for each feature in the test set
    for month in test_months:
        original_month_str = str(month)
        if isinstance(month, str) and len(original_month_str) >= 7:
            original_month_str = original_month_str[:7]  # Extract YYYY-MM part
        
        # Use the prefixed month string for sorting
        month_str = prefix_map[month]
        
        month_df = test_df[test_df['Application_month'] == month]
        
        if not month_df.empty:
            month_accounts = month_df['digitalLoanAccountId'].nunique()
            
            for feature in feature_list:
                if feature not in month_df.columns:
                    continue
                    
                if feature in feature_train_distributions:  # Check if we have a training distribution
                    # Calculate distribution for the current month
                    counts = month_df[feature].value_counts(dropna=True)
                    if len(counts) > 0:  # Ensure we have data for this feature in this month
                        distribution = counts / counts.sum()
                        
                        # Calculate PSI comparing with training distribution
                        month_psi = calculate_categorical_psi(
                            feature_train_distributions[feature],
                            distribution
                        )
                        
                        psi_results.append({
                            'Month': original_month_str,  # Original month string for display
                            'MonthSortKey': month_str,    # Prefixed month string for sorting
                            'feature': feature,
                            'DateCategory': 'b_Monthly',  # Added 'b_' prefix for sorting
                            'csivalues': month_psi,
                            'account_count': month_accounts
                        })
    
    # Create the output DataFrame
    return pd.DataFrame(psi_results)

# Features list
feature_list = [
    'beta_de_ln_vas_opted_flag_bin',
    'beta_de_ln_doc_type_rolled_bin',
    'beta_de_ln_marital_status_bin',
    'beta_de_ln_age_bin',
    'beta_de_ln_ref1_type_bin'
]

# Define segment columns
segment_columns = ['ln_user_type', 'ln_prod_type', 'ln_os_type']  # Changed ln_os_type to ln_prod_type based on your data

# Calculate PSI for overall and by segments
all_results = calculate_segmented_psi(sildemodf, feature_list, segment_columns)

# Combine with s_apps_score results if they exist
try:
    # First ensure the s_apps_score_output_df has the same structure
    if 'MonthSortKey' not in s_apps_score_output_df.columns:
        s_apps_score_output_df['MonthSortKey'] = s_apps_score_output_df['Month']
        # Update DateCategory with prefix
        s_apps_score_output_df['DateCategory'] = s_apps_score_output_df['DateCategory'].apply(
            lambda x: 'a_Training' if x == 'Training' else 'b_Monthly'
        )
    
    # Add segment info to s_apps_score_output_df
    s_apps_score_output_df['segment_type'] = 'Overall'
    s_apps_score_output_df['segment_value'] = 'All'
    
    # Check if s_apps_score_output_df exists
    combined_results = pd.concat([s_apps_score_output_df, all_results], ignore_index=True)
    
    # Replace 'scorename' with 'feature' for consistency
    if 'scorename' in combined_results.columns:
        combined_results['feature'] = combined_results['feature'].fillna(combined_results['scorename'])
        combined_results = combined_results.drop('scorename', axis=1)
except NameError:
    # If s_apps_score_output_df doesn't exist, just use all_results
    combined_results = all_results

# Sort by segment_type, segment_value, MonthSortKey and feature
sort_columns = ['segment_type', 'segment_value']
if 'MonthSortKey' in combined_results.columns:
    sort_columns.append('MonthSortKey')
else:
    sort_columns.append('Month')
sort_columns.append('feature')

combined_results = combined_results.sort_values(sort_columns)

# Save the results
combined_results.to_csv('segmented_feature_psi_results_Beta_Demo.csv', index=False)

# Display the first few rows
print("Sample of results:")
print(combined_results.head())

# Create pivot tables for easier analysis
print("\nGenerating pivot tables for analysis...")

# Function to create pivot table for a given segment
def create_pivot_for_segment(data, segment_type, segment_value):
    segment_data = data[(data['segment_type'] == segment_type) & 
                       (data['segment_value'] == segment_value)]
    
    # Create pivot table
    pivot = segment_data.pivot_table(
        index=['feature'], 
        columns=['MonthSortKey'] if 'MonthSortKey' in segment_data.columns else ['Month'], 
        values='csivalues',
        aggfunc='first'
    )
    
    return pivot

# Create and save pivot tables for overall and each segment
unique_segments = combined_results[['segment_type', 'segment_value']].drop_duplicates()

# Create a dictionary to store all pivot tables
pivot_tables = {}

# Generate pivot tables for each segment
for _, row in unique_segments.iterrows():
    segment_type = row['segment_type']
    segment_value = row['segment_value']
    
    # Create a key for this segment
    segment_key = f"{segment_type}_{segment_value}"
    
    # Create pivot table
    pivot = create_pivot_for_segment(combined_results, segment_type, segment_value)
    
    # Store in dictionary
    pivot_tables[segment_key] = pivot
    
    # Print some info
    print(f"\nPivot table for {segment_type} - {segment_value}:")
    print(pivot.head(3))  # Show just first 3 rows

# Save all pivot tables to Excel with multiple sheets
with pd.ExcelWriter('segmented_feature_psi_pivots_beta_demo.xlsx') as writer:
    for segment_key, pivot_df in pivot_tables.items():
        # Clean sheet name (Excel has 31 character limit)
        sheet_name = segment_key[:31].replace('/', '_')
        pivot_df.to_excel(writer, sheet_name=sheet_name)

print("\nAll pivot tables saved to 'segmented_feature_psi_pivots_beta_demo.xlsx'")

Sample of results:
     Month                         feature DateCategory  csivalues  \
8  2024-07              beta_de_ln_age_bin    b_Monthly   0.003228   
6  2024-07  beta_de_ln_doc_type_rolled_bin    b_Monthly   0.021654   
7  2024-07   beta_de_ln_marital_status_bin    b_Monthly   0.000437   
9  2024-07        beta_de_ln_ref1_type_bin    b_Monthly   0.736611   
5  2024-07   beta_de_ln_vas_opted_flag_bin    b_Monthly   0.022247   

   account_count MonthSortKey segment_type segment_value  
8          20382    b_2024-07      Overall           All  
6          20382    b_2024-07      Overall           All  
7          20382    b_2024-07      Overall           All  
9          20382    b_2024-07      Overall           All  
5          20382    b_2024-07      Overall           All  

Generating pivot tables for analysis...

Pivot table for Overall - All:
MonthSortKey                    b_2024-07  c_2024-08  d_2024-09  e_2024-10  \
feature                                                

In [23]:
combined_results['MonthSortKey'].value_counts(dropna=False)

MonthSortKey
b_2024-07    45
c_2024-08    45
d_2024-09    45
e_2024-10    45
f_2024-11    45
g_2024-12    45
h_2025-01    45
i_2025-02    45
j_2025-03    45
NaN          45
Name: count, dtype: int64

In [24]:
combined_results['Month'] = combined_results['Month'].replace('2024-06', '2023-07-2024-06')
combined_results['MonthSortKey'] = combined_results['MonthSortKey'].fillna('a_2023-07-2024-06')
combined_results['Month'] = combined_results['Month'].apply(lambda x: x.split(' 00:00:00')[0] if'00:00:00' in x else x)
combined_results['scorename'] = 'beta_demo_score'
combined_results['Modelname'] = 'SIL Beta Demo'
combined_results['Description'] = 'Train period from 2023-07 to 2024-06'
combined_results

,Month,feature,DateCategory,csivalues,account_count,MonthSortKey,segment_type,segment_value,scorename,Modelname,Description
8,2024-07,beta_de_ln_age_bin,b_Monthly,0.003228,20382,b_2024-07,Overall,All,beta_demo_score,SIL Beta Demo,Train period from 2023-07 to 2024-06
6,2024-07,beta_de_ln_doc_type_rolled_bin,b_Monthly,0.021654,20382,b_2024-07,Overall,All,beta_demo_score,SIL Beta Demo,Train period from 2023-07 to 2024-06
7,2024-07,beta_de_ln_marital_status_bin,b_Monthly,0.000437,20382,b_2024-07,Overall,All,beta_demo_score,SIL Beta Demo,Train period from 2023-07 to 2024-06
9,2024-07,beta_de_ln_ref1_type_bin,b_Monthly,0.736611,20382,b_2024-07,Overall,All,beta_demo_score,SIL Beta Demo,Train period from 2023-07 to 2024-06
5,2024-07,beta_de_ln_vas_opted_flag_bin,b_Monthly,0.022247,20382,b_2024-07,Overall,All,beta_demo_score,SIL Beta Demo,Train period from 2023-07 to 2024-06
...,...,...,...,...,...,...,...,...,...,...,...
53,2023-07-2024-06,beta_de_ln_age_bin,a_Training,0.000000,50154,a_2023-07-2024-06,ln_user_type,3_Applied_Not_Disbursed,beta_demo_score,SIL Beta Demo,Train period from 2023-07 to 2024-06
51,2023-07-2024-06,beta_de_ln_doc_type_rolled_bin,a_Training,0.000000,50154,a_2023-07-2024-06,ln_user_type,3_Applied_Not_Disbursed,beta_demo_score,SIL Beta Demo,Train period from 2023-07 to 2024-06
52,2023-07-2024-06,beta_de_ln_marital_status_bin,a_Training,0.000000,50154,a_2023-07-2024-06,ln_user_type,3_Applied_Not_Disbursed,beta_demo_score,SIL Beta Demo,Train period from 2023-07 to 2024-06
54,2023-07-2024-06,beta_de_ln_ref1_type_bin,a_Training,0.000000,50154,a_2023-07-2024-06,ln_user_type,3_Applied_Not_Disbursed,beta_demo_score,SIL Beta Demo,Train period from 2023-07 to 2024-06


In [25]:
dataset_id = 'dap_ds_poweruser_playground'
table_id = 'F_CSI_TT_NEW'
# Define the table schema as per your DataFrame columns
schema = [
    bigquery.SchemaField("Month", "string"),
    bigquery.SchemaField("feature", "string"),
    bigquery.SchemaField("DateCategory", "string"),
    bigquery.SchemaField("csivalues", "float64"),
    bigquery.SchemaField("account_count", "int64"),
    bigquery.SchemaField("MonthSortKey", "string"),
    bigquery.SchemaField("segment_type", "string"),
    bigquery.SchemaField("segment_value", "string"),
    bigquery.SchemaField("scorename", "string"),
    bigquery.SchemaField("Modelname", "string"),
    bigquery.SchemaField("Description", "string")
    ]
# Create the dataset reference
dataset_ref = client.dataset(dataset_id)
# Define the table reference
table_ref = dataset_ref.table(table_id)
# Configure the job to overwrite the table if it already exists
job_config = bigquery.LoadJobConfig(schema = schema)
# Load the DataFrame into BigQuery
job = client.load_table_from_dataframe(combined_results, table_ref, job_config=job_config)
# Wait for the job to complete
job.result()
print(f"Table {table_id} created in dataset {dataset_id}.")

Table F_CSI_TT_NEW created in dataset dap_ds_poweruser_playground.


In [26]:
sq = """
with appscore as 
(select 
digitalLoanAccountId,
FORMAT_DATE('%Y-%m', ln_appln_submit_datetime) Application_month,
FORMAT_DATE('%F', DATE_TRUNC(ln_appln_submit_datetime, WEEK(MONDAY))) as Appl_week_start_date,
EXTRACT(WEEK(MONDAY) FROM ln_appln_submit_datetime) as Appl_week_number,
ln_user_type,
ln_loan_type,
ln_prod_type, 
ln_os_type,
apps_score,
case when date_trunc(ln_appln_submit_datetime, day) between '2023-12-01' and '2024-06-30' then 'Train'
       when date_trunc(ln_appln_submit_datetime, day) >= '2024-07-01' then 'Test'
       Else 'Other' end dataselection,
case when app_first_competitors_install_to_apply_days is null then 'k. missing'
     when app_first_competitors_install_to_apply_days between -2.001 and 4.1 then 'a. -2.001-4.1'
     when app_first_competitors_install_to_apply_days between 4.2 and 40.1 then 'b. 4.2-40.1'
     when app_first_competitors_install_to_apply_days between 40.2 and 88.4 then 'c. 40.2-88.4'
     when app_first_competitors_install_to_apply_days between 88.5 and 143.2 then 'd. 88.5-143.2'
     when app_first_competitors_install_to_apply_days between 143.3 and 206.7 then 'e 143.3-206.7'
     when app_first_competitors_install_to_apply_days between 206.8 and 288.3 then 'f. 206.8-288.2'
     when app_first_competitors_install_to_apply_days between 288.4 and 391.9 then 'g. 288.4-391.9'
     when app_first_competitors_install_to_apply_days between 392.0 and 547.3 then 'h. 392.0-547.3'
     when app_first_competitors_install_to_apply_days between 547.4 and 826.1 then 'i. 547.4-826.1'
     when app_first_competitors_install_to_apply_days between 826.2 and 5242.2 then 'j. 826.2-5242.2'
     when app_first_competitors_install_to_apply_days > 5242.2 then 'l. >5242.2' end app_first_competitors_install_to_apply_days_bin,
case 
    when app_median_time_bw_installed_mins_30d is null then 'l. missing'
    when app_median_time_bw_installed_mins_30d between -0.001 and 96.022 then 'a. -0.001-96.022'
    when app_median_time_bw_installed_mins_30d between 96.023 and 1166.377 then 'b. 96.023-1166.377'
    when app_median_time_bw_installed_mins_30d between 1166.378 and 2259.803 then 'c. 1166.378-2259.803'
    when app_median_time_bw_installed_mins_30d between 2259.804 and 3532.3 then 'd. 2259.804-3532.3'
    when app_median_time_bw_installed_mins_30d between 3532.4 and 5067.042 then 'e. 3532.4-5067.042'
    when app_median_time_bw_installed_mins_30d between 5067.043 and 7065.507 then 'f. 5067.043-7065.507'
    when app_median_time_bw_installed_mins_30d between 7065.508 and 9891.612 then 'g. 7065.508-9891.612'
    when app_median_time_bw_installed_mins_30d between 9891.613 and 14384.46 then 'h. 9891.613-14384.46'
    when app_median_time_bw_installed_mins_30d between 14384.461 and 20358.378 then 'j. 14384.461-20358.378'
    when app_median_time_bw_installed_mins_30d between 20358.379 and 112663145.6 then 'k. 20358.379-112663145.6'
    else 'm. NA'end app_median_time_bw_installed_mins_30d_bin,
CASE
    WHEN app_cnt_absence_tag_90d is null then 'g. missing'
    WHEN app_cnt_absence_tag_90d BETWEEN -0.001 AND 1.0 THEN 'a. (-0.001, 1.0]'
    WHEN app_cnt_absence_tag_90d BETWEEN 1.001 AND 2.0 THEN 'b. (1.0, 2.0]'
    WHEN app_cnt_absence_tag_90d BETWEEN 2.001 AND 3.0 THEN 'c. (2.0, 3.0]'
    WHEN app_cnt_absence_tag_90d BETWEEN 3.001 AND 4.0 THEN 'd. (3.0, 4.0]'
    WHEN app_cnt_absence_tag_90d BETWEEN 4.001 AND 7.0 THEN 'e. (4.0, 7.0]'
    WHEN app_cnt_absence_tag_90d BETWEEN 7.001 AND 154.0 THEN 'f. (7.0, 154.0]'
    ELSE 'h. NA' END app_cnt_absence_tag_90d_bin,
CASE
    WHEN app_cnt_finance_90d is null then 'f. missing'
    WHEN app_cnt_finance_90d BETWEEN -0.001 AND 1.0 THEN 'a. (-0.001, 1.0]'
    WHEN app_cnt_finance_90d BETWEEN 1.001 AND 2.0 THEN 'b. (1.0, 2.0]'
    WHEN app_cnt_finance_90d BETWEEN 2.001 AND 3.0 THEN 'c. (2.0, 3.0]'
    WHEN app_cnt_finance_90d BETWEEN 3.001 AND 4.0 THEN 'd. (3.0, 4.0]'
    WHEN app_cnt_finance_90d BETWEEN 4.001 AND 30.0 THEN 'e. (4.0, 30.0]'
    ELSE 'g. NA'  
END app_cnt_finance_90d_bin,
CASE
    WHEN app_first_payday_install_to_apply_days is null then 'h. missing'
    WHEN app_first_payday_install_to_apply_days BETWEEN -1.001 AND 0.0 THEN 'a. (-1.001, 0.0]'
    WHEN app_first_payday_install_to_apply_days BETWEEN 0.001 AND 0.1 THEN 'b. (0.0, 0.1]'
    WHEN app_first_payday_install_to_apply_days BETWEEN 0.101 AND 42.1 THEN 'c. (0.1, 42.1]'
    WHEN app_first_payday_install_to_apply_days BETWEEN 42.101 AND 138.8 THEN 'd. (42.1, 138.8]'
    WHEN app_first_payday_install_to_apply_days BETWEEN 138.801 AND 273.54 THEN 'e. (138.8, 273.54]'
    WHEN app_first_payday_install_to_apply_days BETWEEN 273.541 AND 532.2 THEN 'f. (273.54, 532.2]'
    WHEN app_first_payday_install_to_apply_days BETWEEN 532.201 AND 5242.2 THEN 'g. (532.2, 5242.2]'
    ELSE 'i. NA'  -- Add an else clause to handle values outside the defined ranges
END app_first_payday_install_to_apply_days_bin
from risk_mart.sil_risk_ds_master_20230101_20250309
where date_trunc(ln_appln_submit_datetime, day) >= '2023-12-01'
)
select * from appscore;
"""

appscoredf = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID e0ac1038-57d4-46c3-b363-5986feb56a17 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [27]:
print(f"The shape of the appscoredf dataframe is :\t {appscoredf.shape}")

The shape of the appscoredf dataframe is :	 (315412, 15)


In [28]:
appscoredf.head()

,digitalLoanAccountId,Application_month,Appl_week_start_date,Appl_week_number,ln_user_type,ln_loan_type,ln_prod_type,ln_os_type,apps_score,dataselection,app_first_competitors_install_to_apply_days_bin,app_median_time_bw_installed_mins_30d_bin,app_cnt_absence_tag_90d_bin,app_cnt_finance_90d_bin,app_first_payday_install_to_apply_days_bin
0,e868d340-7612-45e3-8326-a88785cd6320,2023-12,2023-11-27,48,3_Applied_Not_Disbursed,SIL-Instore,Appliance,iOS,NaN,Train,k. missing,l. missing,g. missing,f. missing,h. missing
1,8effa90f-b747-40b9-b42f-39691224f596,2023-12,2023-11-27,48,2_New Applicant,SIL-Instore,Mall,Android,0.742207,Train,d. 88.5-143.2,a. -0.001-96.022,"f. (7.0, 154.0]","e. (4.0, 30.0]","d. (42.1, 138.8]"
2,316ddfea-89b4-40ff-b6ae-21e7fc462a79,2023-12,2023-11-27,48,2_New Applicant,SIL-Instore,Appliance,iOS,NaN,Train,k. missing,l. missing,g. missing,f. missing,h. missing
3,1bb96350-57cd-4d78-8485-036aac9517f8,2023-12,2023-11-27,48,2_New Applicant,SIL-Instore,Mall,Android,0.688786,Train,c. 40.2-88.4,d. 2259.804-3532.3,"e. (4.0, 7.0]","e. (4.0, 30.0]","d. (42.1, 138.8]"
4,222882e3-a168-4a45-85fb-c27cc7a549cd,2023-12,2023-11-27,48,1_Repeat Applicant,SIL-Instore,Mall,Android,0.520746,Train,c. 40.2-88.4,j. 14384.461-20358.378,"b. (1.0, 2.0]","b. (1.0, 2.0]","a. (-1.001, 0.0]"


In [29]:
appscoredf.columns

Index(['digitalLoanAccountId', 'Application_month', 'Appl_week_start_date',
       'Appl_week_number', 'ln_user_type', 'ln_loan_type', 'ln_prod_type',
       'ln_os_type', 'apps_score', 'dataselection',
       'app_first_competitors_install_to_apply_days_bin',
       'app_median_time_bw_installed_mins_30d_bin',
       'app_cnt_absence_tag_90d_bin', 'app_cnt_finance_90d_bin',
       'app_first_payday_install_to_apply_days_bin'],
      dtype='object')

In [30]:
import pandas as pd
import numpy as np
from datetime import datetime

def calculate_categorical_psi(train_dist, test_dist):
    """
    Calculate PSI for categorical features.
    
    Args:
        train_dist: Distribution of categories in training set
        test_dist: Distribution of categories in test set
    
    Returns:
        float: PSI value
    """
    # Ensure both distributions have the same categories
    all_categories = set(train_dist.index) | set(test_dist.index)
    
    # Align distributions
    train_dist_aligned = train_dist.reindex(all_categories, fill_value=0.0001)  # Small value to avoid division by zero
    test_dist_aligned = test_dist.reindex(all_categories, fill_value=0.0001)
    
    # Calculate PSI
    psi_values = (test_dist_aligned - train_dist_aligned) * np.log(test_dist_aligned / train_dist_aligned)
    return psi_values.sum()

def calculate_segmented_psi(df, feature_list, segment_columns=None):
    """
    Calculate PSI for multiple categorical features, overall and by segments.
    
    Args:
        df: DataFrame containing the data
        feature_list: List of feature names to calculate PSI for
        segment_columns: List of columns to segment by (e.g., ['ln_user_type', 'ln_os_type'])
    
    Returns:
        DataFrame: PSI results for each feature by month and segment
    """
    # Initialize results list
    all_results = []
    
    # If no segment columns are provided, use an empty list
    if segment_columns is None:
        segment_columns = []
    
    # First, calculate overall PSI
    overall_results = calculate_feature_psi(df, feature_list)
    overall_results['segment_type'] = 'Overall'
    overall_results['segment_value'] = 'All'
    all_results.append(overall_results)
    
    # Then calculate PSI for each segment column
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            print(f"Warning: {segment_col} not found in DataFrame. Skipping.")
            continue
        
        # Get unique segment values
        segment_values = df[segment_col].dropna().unique()
        
        for segment_val in segment_values:
            # Filter data for this segment
            segment_df = df[df[segment_col] == segment_val]
            
            # Skip if not enough data
            if len(segment_df) < 50:  # Arbitrary threshold
                print(f"Skipping {segment_col}={segment_val} due to insufficient data ({len(segment_df)} rows).")
                continue
                
            # Calculate PSI for this segment
            try:
                segment_results = calculate_feature_psi(segment_df, feature_list)
                segment_results['segment_type'] = segment_col
                segment_results['segment_value'] = segment_val
                all_results.append(segment_results)
            except Exception as e:
                print(f"Error calculating PSI for {segment_col}={segment_val}: {e}")
    
    # Combine all results
    if all_results:
        combined_results = pd.concat(all_results, ignore_index=True)
        return combined_results
    else:
        return pd.DataFrame()

def calculate_feature_psi(df, feature_list):
    """
    Calculate PSI for multiple categorical features.
    
    Args:
        df: DataFrame containing the data
        feature_list: List of feature names to calculate PSI for
    
    Returns:
        DataFrame: PSI results for each feature by month
    """
    # Make a copy to avoid modifying the original DataFrame
    df_copy = df.copy()
    
    # Separate train and test data
    train_df = df_copy[df_copy['dataselection'] == 'Train']
    test_df = df_copy[df_copy['dataselection'] == 'Test']
    
    # Skip if either dataset is empty
    if train_df.empty or test_df.empty:
        print("Warning: Either train or test dataset is empty. Skipping PSI calculation.")
        return pd.DataFrame()
    
    # Handle Application_month based on its type
    if isinstance(df_copy['Application_month'].iloc[0], str):
        # If it's a string in format 'YYYY-MM-DD', extract just 'YYYY-MM'
        last_train_month_str = str(train_df['Application_month'].max())
        if len(last_train_month_str) >= 7:  # Ensure we have at least YYYY-MM
            last_train_month_str = last_train_month_str[:7]  # Extract YYYY-MM part
    else:
        # If it's already a datetime object
        try:
            last_train_month = pd.to_datetime(train_df['Application_month'].max())
            last_train_month_str = last_train_month.strftime('%Y-%m')
        except:
            # Fallback if conversion fails
            last_train_month_str = str(train_df['Application_month'].max())
    
    # Store PSI results
    psi_results = []
    
    # Calculate distribution for each feature in the training set
    feature_train_distributions = {}
    for feature in feature_list:
        if feature not in train_df.columns:
            print(f"Warning: Feature {feature} not found in training data. Skipping.")
            continue
            
        # Calculate distribution, excluding missing values
        counts = train_df[feature].value_counts(dropna=True)
        if len(counts) > 0:  # Check if we have any non-null values
            distribution = counts / counts.sum()
            feature_train_distributions[feature] = distribution
            
            # Add the train set PSI (which is 0 by definition when comparing with itself)
            psi_results.append({
                'Month': last_train_month_str,
                'feature': feature,
                'DateCategory': 'a_Training',  # Added 'a_' prefix for sorting
                'csivalues': 0.0,  # PSI of training set against itself is 0
                'account_count': train_df['digitalLoanAccountId'].nunique()
            })
    
    # Get unique months from test set and sort them
    test_months = sorted(test_df['Application_month'].unique())
    
    # Create mapping of months to prefixed labels (b, c, d, etc.)
    prefix_map = {}
    for i, month in enumerate(test_months):
        month_str = str(month)
        if isinstance(month, str) and len(month_str) >= 7:
            month_str = month_str[:7]  # Extract YYYY-MM part
        
        # Use letters b, c, d, etc. for subsequent months (a is reserved for Training)
        prefix = chr(98 + i)  # ASCII: b=98, c=99, etc.
        prefix_map[month] = f"{prefix}_{month_str}"
    
    # Calculate monthly PSI for each feature in the test set
    for month in test_months:
        original_month_str = str(month)
        if isinstance(month, str) and len(original_month_str) >= 7:
            original_month_str = original_month_str[:7]  # Extract YYYY-MM part
        
        # Use the prefixed month string for sorting
        month_str = prefix_map[month]
        
        month_df = test_df[test_df['Application_month'] == month]
        
        if not month_df.empty:
            month_accounts = month_df['digitalLoanAccountId'].nunique()
            
            for feature in feature_list:
                if feature not in month_df.columns:
                    continue
                    
                if feature in feature_train_distributions:  # Check if we have a training distribution
                    # Calculate distribution for the current month
                    counts = month_df[feature].value_counts(dropna=True)
                    if len(counts) > 0:  # Ensure we have data for this feature in this month
                        distribution = counts / counts.sum()
                        
                        # Calculate PSI comparing with training distribution
                        month_psi = calculate_categorical_psi(
                            feature_train_distributions[feature],
                            distribution
                        )
                        
                        psi_results.append({
                            'Month': original_month_str,  # Original month string for display
                            'MonthSortKey': month_str,    # Prefixed month string for sorting
                            'feature': feature,
                            'DateCategory': 'b_Monthly',  # Added 'b_' prefix for sorting
                            'csivalues': month_psi,
                            'account_count': month_accounts
                        })
    
    # Create the output DataFrame
    return pd.DataFrame(psi_results)

# Features list
feature_list = [
  'app_first_competitors_install_to_apply_days_bin',
       'app_median_time_bw_installed_mins_30d_bin',
       'app_cnt_absence_tag_90d_bin',
       'app_cnt_finance_90d_bin',
       'app_first_payday_install_to_apply_days_bin'
]

# Define segment columns
segment_columns = ['ln_user_type', 'ln_prod_type', 'ln_os_type']  # Changed ln_os_type to ln_prod_type based on your data

# Calculate PSI for overall and by segments
all_results = calculate_segmented_psi(appscoredf, feature_list, segment_columns)

# Combine with s_apps_score results if they exist
try:
    # First ensure the s_apps_score_output_df has the same structure
    if 'MonthSortKey' not in s_apps_score_output_df.columns:
        s_apps_score_output_df['MonthSortKey'] = s_apps_score_output_df['Month']
        # Update DateCategory with prefix
        s_apps_score_output_df['DateCategory'] = s_apps_score_output_df['DateCategory'].apply(
            lambda x: 'a_Training' if x == 'Training' else 'b_Monthly'
        )
    
    # Add segment info to s_apps_score_output_df
    s_apps_score_output_df['segment_type'] = 'Overall'
    s_apps_score_output_df['segment_value'] = 'All'
    
    # Check if s_apps_score_output_df exists
    combined_results = pd.concat([s_apps_score_output_df, all_results], ignore_index=True)
    
    # Replace 'scorename' with 'feature' for consistency
    if 'scorename' in combined_results.columns:
        combined_results['feature'] = combined_results['feature'].fillna(combined_results['scorename'])
        combined_results = combined_results.drop('scorename', axis=1)
except NameError:
    # If s_apps_score_output_df doesn't exist, just use all_results
    combined_results = all_results

# Sort by segment_type, segment_value, MonthSortKey and feature
sort_columns = ['segment_type', 'segment_value']
if 'MonthSortKey' in combined_results.columns:
    sort_columns.append('MonthSortKey')
else:
    sort_columns.append('Month')
sort_columns.append('feature')

combined_results = combined_results.sort_values(sort_columns)

# Save the results
combined_results.to_csv('segmented_feature_psi_pivots_app_score.csv', index=False)

# Display the first few rows
print("Sample of results:")
print(combined_results.head())

# Create pivot tables for easier analysis
print("\nGenerating pivot tables for analysis...")

# Function to create pivot table for a given segment
def create_pivot_for_segment(data, segment_type, segment_value):
    segment_data = data[(data['segment_type'] == segment_type) & 
                       (data['segment_value'] == segment_value)]
    
    # Create pivot table
    pivot = segment_data.pivot_table(
        index=['feature'], 
        columns=['MonthSortKey'] if 'MonthSortKey' in segment_data.columns else ['Month'], 
        values='csivalues',
        aggfunc='first'
    )
    
    return pivot

# Create and save pivot tables for overall and each segment
unique_segments = combined_results[['segment_type', 'segment_value']].drop_duplicates()

# Create a dictionary to store all pivot tables
pivot_tables = {}

# Generate pivot tables for each segment
for _, row in unique_segments.iterrows():
    segment_type = row['segment_type']
    segment_value = row['segment_value']
    
    # Create a key for this segment
    segment_key = f"{segment_type}_{segment_value}"
    
    # Create pivot table
    pivot = create_pivot_for_segment(combined_results, segment_type, segment_value)
    
    # Store in dictionary
    pivot_tables[segment_key] = pivot
    
    # Print some info
    print(f"\nPivot table for {segment_type} - {segment_value}:")
    print(pivot.head(3))  # Show just first 3 rows

# Save all pivot tables to Excel with multiple sheets
with pd.ExcelWriter('segmented_feature_psi_pivots_app_score.xlsx') as writer:
    for segment_key, pivot_df in pivot_tables.items():
        # Clean sheet name (Excel has 31 character limit)
        sheet_name = segment_key[:31].replace('/', '_')
        pivot_df.to_excel(writer, sheet_name=sheet_name)

print("\nAll pivot tables saved to 'segmented_feature_psi_pivots_app_score.xlsx'")

Sample of results:
     Month                                          feature DateCategory  \
7  2024-07                      app_cnt_absence_tag_90d_bin    b_Monthly   
8  2024-07                          app_cnt_finance_90d_bin    b_Monthly   
5  2024-07  app_first_competitors_install_to_apply_days_bin    b_Monthly   
9  2024-07       app_first_payday_install_to_apply_days_bin    b_Monthly   
6  2024-07        app_median_time_bw_installed_mins_30d_bin    b_Monthly   

   csivalues  account_count MonthSortKey segment_type segment_value  
7   0.001068          20382    b_2024-07      Overall           All  
8   0.002306          20382    b_2024-07      Overall           All  
5   0.002313          20382    b_2024-07      Overall           All  
9   0.001069          20382    b_2024-07      Overall           All  
6   0.004607          20382    b_2024-07      Overall           All  

Generating pivot tables for analysis...

Pivot table for Overall - All:
MonthSortKey                   

In [31]:
combined_results

,Month,feature,DateCategory,csivalues,account_count,MonthSortKey,segment_type,segment_value
7,2024-07,app_cnt_absence_tag_90d_bin,b_Monthly,0.001068,20382,b_2024-07,Overall,All
8,2024-07,app_cnt_finance_90d_bin,b_Monthly,0.002306,20382,b_2024-07,Overall,All
5,2024-07,app_first_competitors_install_to_apply_days_bin,b_Monthly,0.002313,20382,b_2024-07,Overall,All
9,2024-07,app_first_payday_install_to_apply_days_bin,b_Monthly,0.001069,20382,b_2024-07,Overall,All
6,2024-07,app_median_time_bw_installed_mins_30d_bin,b_Monthly,0.004607,20382,b_2024-07,Overall,All
...,...,...,...,...,...,...,...,...
52,2024-06,app_cnt_absence_tag_90d_bin,a_Training,0.000000,39369,NaN,ln_user_type,3_Applied_Not_Disbursed
53,2024-06,app_cnt_finance_90d_bin,a_Training,0.000000,39369,NaN,ln_user_type,3_Applied_Not_Disbursed
50,2024-06,app_first_competitors_install_to_apply_days_bin,a_Training,0.000000,39369,NaN,ln_user_type,3_Applied_Not_Disbursed
54,2024-06,app_first_payday_install_to_apply_days_bin,a_Training,0.000000,39369,NaN,ln_user_type,3_Applied_Not_Disbursed


In [32]:
combined_results['Month'] = combined_results['Month'].replace('2024-06', '2023-12-2024-06')
combined_results['MonthSortKey'] = combined_results['MonthSortKey'].fillna('a_2023-12-2024-06')
combined_results['Month'] = combined_results['Month'].apply(lambda x: x.split(' 00:00:00')[0] if'00:00:00' in x else x)
combined_results['scorename'] = 'apps_score'
combined_results['Modelname'] = 'Android_SIL_Apps_Score'
combined_results['Description'] = 'Train period from 2023-12 to 2024-06'
combined_results

,Month,feature,DateCategory,csivalues,account_count,MonthSortKey,segment_type,segment_value,scorename,Modelname,Description
7,2024-07,app_cnt_absence_tag_90d_bin,b_Monthly,0.001068,20382,b_2024-07,Overall,All,apps_score,Android_SIL_Apps_Score,Train period from 2023-12 to 2024-06
8,2024-07,app_cnt_finance_90d_bin,b_Monthly,0.002306,20382,b_2024-07,Overall,All,apps_score,Android_SIL_Apps_Score,Train period from 2023-12 to 2024-06
5,2024-07,app_first_competitors_install_to_apply_days_bin,b_Monthly,0.002313,20382,b_2024-07,Overall,All,apps_score,Android_SIL_Apps_Score,Train period from 2023-12 to 2024-06
9,2024-07,app_first_payday_install_to_apply_days_bin,b_Monthly,0.001069,20382,b_2024-07,Overall,All,apps_score,Android_SIL_Apps_Score,Train period from 2023-12 to 2024-06
6,2024-07,app_median_time_bw_installed_mins_30d_bin,b_Monthly,0.004607,20382,b_2024-07,Overall,All,apps_score,Android_SIL_Apps_Score,Train period from 2023-12 to 2024-06
...,...,...,...,...,...,...,...,...,...,...,...
52,2023-12-2024-06,app_cnt_absence_tag_90d_bin,a_Training,0.000000,39369,a_2023-12-2024-06,ln_user_type,3_Applied_Not_Disbursed,apps_score,Android_SIL_Apps_Score,Train period from 2023-12 to 2024-06
53,2023-12-2024-06,app_cnt_finance_90d_bin,a_Training,0.000000,39369,a_2023-12-2024-06,ln_user_type,3_Applied_Not_Disbursed,apps_score,Android_SIL_Apps_Score,Train period from 2023-12 to 2024-06
50,2023-12-2024-06,app_first_competitors_install_to_apply_days_bin,a_Training,0.000000,39369,a_2023-12-2024-06,ln_user_type,3_Applied_Not_Disbursed,apps_score,Android_SIL_Apps_Score,Train period from 2023-12 to 2024-06
54,2023-12-2024-06,app_first_payday_install_to_apply_days_bin,a_Training,0.000000,39369,a_2023-12-2024-06,ln_user_type,3_Applied_Not_Disbursed,apps_score,Android_SIL_Apps_Score,Train period from 2023-12 to 2024-06


In [33]:
dataset_id = 'dap_ds_poweruser_playground'
table_id = 'F_CSI_TT_NEW'
# Define the table schema as per your DataFrame columns
schema = [
    bigquery.SchemaField("Month", "string"),
    bigquery.SchemaField("feature", "string"),
    bigquery.SchemaField("DateCategory", "string"),
    bigquery.SchemaField("csivalues", "float64"),
    bigquery.SchemaField("account_count", "int64"),
    bigquery.SchemaField("MonthSortKey", "string"),
    bigquery.SchemaField("segment_type", "string"),
    bigquery.SchemaField("segment_value", "string"),
    bigquery.SchemaField("scorename", "string"),
    bigquery.SchemaField("Modelname", "string"),
    bigquery.SchemaField("Description", "string")
    ]
# Create the dataset reference
dataset_ref = client.dataset(dataset_id)
# Define the table reference
table_ref = dataset_ref.table(table_id)
# Configure the job to overwrite the table if it already exists
job_config = bigquery.LoadJobConfig(schema = schema)
# Load the DataFrame into BigQuery
job = client.load_table_from_dataframe(combined_results, table_ref, job_config=job_config)
# Wait for the job to complete
job.result()
print(f"Table {table_id} created in dataset {dataset_id}.")

Table F_CSI_TT_NEW created in dataset dap_ds_poweruser_playground.
